In [18]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tabulate import tabulate

warnings.filterwarnings('ignore')


# Настройка отображения
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [2]:
df = pd.read_csv("kc_house_data.csv")

df.head(8)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,7237550310,20140512T000000,1225000.0,4,4.50,5420,101930,1.0,0,0,3,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,2008000270,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,3,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711


### Удаление выбросов

In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv('kc_house_data.csv')

print(f"Исходное количество строк: {len(df)}")

# Функция для удаления выбросов
def remove_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

# Применение функции к количественным признакам
numeric_cols = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot']
df_clean = remove_outliers(df, numeric_cols)

print(f"Количество строк после удаления выбросов: {len(df_clean)}")
print(f"Удалено строк: {len(df) - len(df_clean)}")
print(f"Процент удаленных строк: {((len(df) - len(df_clean)) / len(df) * 100):.2f}%")


Исходное количество строк: 21613
Количество строк после удаления выбросов: 17589
Удалено строк: 4024
Процент удаленных строк: 18.62%


### Пропуски

In [5]:
# Анализ пропущенных значений
print("Количество пропусков в каждом столбце:")
print(df_clean.isnull().sum())
print(f"\nОбщее количество пропусков: {df_clean.isnull().sum().sum()}")


Количество пропусков в каждом столбце:
id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

Общее количество пропусков: 0


In [6]:
df_clean

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


### Нормализация

In [7]:
from sklearn.preprocessing import StandardScaler

# Удаление ненужных столбцов
X = df_clean.drop(['id', 'date', 'price', 'zipcode', 'lat', 'long'], axis=1)
y = df_clean['price']

print(f"Количество признаков: {X.shape[1]}")
print(f"Количество наблюдений: {X.shape[0]}")
print(f"\nПризнаки для модели: {list(X.columns)}")

# Стандартизация признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\n" + "="*60)
print("СТАНДАРТИЗАЦИЯ ЗАВЕРШЕНА")
print("="*60)
print(f"Среднее значение признаков после стандартизации: {X_scaled.mean():.10f}")
print(f"Стандартное отклонение признаков: {X_scaled.std():.10f}")


Количество признаков: 15
Количество наблюдений: 17589

Признаки для модели: ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15']

СТАНДАРТИЗАЦИЯ ЗАВЕРШЕНА
Среднее значение признаков после стандартизации: -0.0000000000
Стандартное отклонение признаков: 1.0000000000


In [8]:
X_scaled


array([[-0.36300892, -1.50553919, -1.04079511, ..., -0.18921474,
        -0.91251839, -0.38464596],
       [-0.36300892,  0.35847483,  1.01680949, ...,  5.27091877,
        -0.29170061,  0.11377367],
       [-1.63669011, -1.50553919, -1.64771445, ..., -0.18921474,
         1.53527744,  0.21977242],
       ...,
       [-1.63669011, -1.878342  , -1.27764168, ..., -0.18921474,
        -1.48012322, -1.29753823],
       [-0.36300892,  0.73127764, -0.41907285, ..., -0.18921474,
        -0.78835483, -1.47796162],
       [-1.63669011, -1.878342  , -1.27764168, ..., -0.18921474,
        -1.48012322, -1.46042046]], shape=(17589, 15))

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import pandas as pd

# Применение One-Hot Encoding для категориальных признаков
df_encoded = df_clean.copy()
categorical_cols = ['floors', 'view', 'condition', 'grade', 'bedrooms', 'bathrooms', 'waterfront']
for col in categorical_cols:
    if col in df_encoded.columns:
        dummies = pd.get_dummies(df_encoded[col], drop_first=False, prefix=col)
        df_encoded = df_encoded.drop(col, axis=1).join(dummies)

# Удаление ненужных столбцов
X = df_encoded.drop(['id', 'date', 'price', 'zipcode', 'lat', 'long'], axis=1)
y = df_encoded['price']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")

# Функция для вычисления RMSE
def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Линейная регрессия
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
rmse_lr = calculate_rmse(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

# Гребневая регрессия (α = 3)
ridge_model = Ridge(alpha=3)
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)
rmse_ridge = calculate_rmse(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

# Lasso регрессия (α = 120)
lasso_model = Lasso(alpha=120, max_iter=10000)
lasso_model.fit(X_train, y_train)
y_pred_lasso = lasso_model.predict(X_test)
rmse_lasso = calculate_rmse(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

# Таблица 1: Качество регрессии
results_table = pd.DataFrame({
    'Метод': ['Линейная регрессия', 'Гребневая регрессия (α=3)', 'Lasso (α=120)'],
    'RMSE': [rmse_lr, rmse_ridge, rmse_lasso],
    'R²': [r2_lr, r2_ridge, r2_lasso]
})

print("\n" + "="*60)
print("ТАБЛИЦА 1: КАЧЕСТВО РЕГРЕССИИ")
print("="*60)
print(results_table.to_string(index=False))

results_table.to_excel('table1_quality_metrics.xlsx', index=False, sheet_name='Качество регрессии')

Размер обучающей выборки: (14071, 53)
Размер тестовой выборки: (3518, 53)

ТАБЛИЦА 1: КАЧЕСТВО РЕГРЕССИИ
                    Метод          RMSE       R²
       Линейная регрессия 128529.845857 0.574560
Гребневая регрессия (α=3) 128556.614537 0.574382
            Lasso (α=120) 128784.872411 0.572870


In [10]:
# Выбор признаков для отображения в таблице
selected_features = ['sqft_living', 'sqft_lot', 'waterfront_0', 'sqft_above', 
                     'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15',
                     'floors_1.0', 'floors_1.5', 'floors_2.0', 'floors_2.5', 
                     'floors_3.0', 'floors_3.5']

# Получение коэффициентов для каждой модели
coefficients_lr = []
coefficients_ridge = []
coefficients_lasso = []

for feature in selected_features:
    if feature in X.columns:
        idx = X.columns.get_loc(feature)
        coefficients_lr.append(lr_model.coef_[idx])
        coefficients_ridge.append(ridge_model.coef_[idx])
        coefficients_lasso.append(lasso_model.coef_[idx])
    else:
        # Если признак не найден, ищем альтернативные названия
        alt_feature = feature.replace('_', '#')
        if alt_feature in X.columns:
            idx = X.columns.get_loc(alt_feature)
            coefficients_lr.append(lr_model.coef_[idx])
            coefficients_ridge.append(ridge_model.coef_[idx])
            coefficients_lasso.append(lasso_model.coef_[idx])
        else:
            coefficients_lr.append(0)
            coefficients_ridge.append(0)
            coefficients_lasso.append(0)

# Таблица 2: Коэффициенты регрессии
coefficients_table = pd.DataFrame({
    'Признак': selected_features,
    'Линейная регрессия': coefficients_lr,
    'Гребневая регрессия (α=3)': coefficients_ridge,
    'Lasso (α=120)': coefficients_lasso
})

print("\n" + "="*60)
print("ТАБЛИЦА 2: КОЭФФИЦИЕНТЫ РЕГРЕССИИ")
print("="*60)
print(coefficients_table.to_string(index=False))

# Сохранение таблицы в Excel
coefficients_table.to_excel('table2_regression_coefficients.xlsx', index=False, sheet_name='Коэффициенты')
print("\nТаблица сохранена в файл 'table2_regression_coefficients.xlsx'")

# Выделение значимых и незначимых коэффициентов для всех моделей
print("\n" + "="*60)
print("АНАЛИЗ ЗНАЧИМОСТИ КОЭФФИЦИЕНТОВ")
print("="*60)

threshold = 1000  # Порог значимости

# Анализ для линейной регрессии
print("\n--- ЛИНЕЙНАЯ РЕГРЕССИЯ ---")
significant_lr = coefficients_table[abs(coefficients_table['Линейная регрессия']) > threshold]
insignificant_lr = coefficients_table[abs(coefficients_table['Линейная регрессия']) <= threshold]

print(f"\nЗначимые признаки (|коэффициент| > {threshold}):")
print(significant_lr[['Признак', 'Линейная регрессия']].to_string(index=False))
print(f"\nНезначимые признаки (|коэффициент| <= {threshold}):")
print(insignificant_lr[['Признак', 'Линейная регрессия']].to_string(index=False))

# Анализ для гребневой регрессии
print("\n--- ГРЕБНЕВАЯ РЕГРЕССИЯ (α=3) ---")
significant_ridge = coefficients_table[abs(coefficients_table['Гребневая регрессия (α=3)']) > threshold]
insignificant_ridge = coefficients_table[abs(coefficients_table['Гребневая регрессия (α=3)']) <= threshold]

print(f"\nЗначимые признаки (|коэффициент| > {threshold}):")
print(significant_ridge[['Признак', 'Гребневая регрессия (α=3)']].to_string(index=False))
print(f"\nНезначимые признаки (|коэффициент| <= {threshold}):")
print(insignificant_ridge[['Признак', 'Гребневая регрессия (α=3)']].to_string(index=False))

# Анализ для Lasso регрессии
print("\n--- LASSO РЕГРЕССИЯ (α=120) ---")
significant_lasso = coefficients_table[abs(coefficients_table['Lasso (α=120)']) > threshold]
insignificant_lasso = coefficients_table[abs(coefficients_table['Lasso (α=120)']) <= threshold]

print(f"\nЗначимые признаки (|коэффициент| > {threshold}):")
print(significant_lasso[['Признак', 'Lasso (α=120)']].to_string(index=False))
print(f"\nНезначимые признаки (|коэффициент| <= {threshold}):")
print(insignificant_lasso[['Признак', 'Lasso (α=120)']].to_string(index=False))

# Сохранение анализа значимости в отдельный файл Excel
with pd.ExcelWriter('table3_significance_analysis.xlsx') as writer:
    significant_lr.to_excel(writer, sheet_name='ЛР_Значимые', index=False)
    insignificant_lr.to_excel(writer, sheet_name='ЛР_Незначимые', index=False)
    significant_ridge.to_excel(writer, sheet_name='Ridge_Значимые', index=False)
    insignificant_ridge.to_excel(writer, sheet_name='Ridge_Незначимые', index=False)
    significant_lasso.to_excel(writer, sheet_name='Lasso_Значимые', index=False)
    insignificant_lasso.to_excel(writer, sheet_name='Lasso_Незначимые', index=False)

print("\nАнализ значимости сохранён в файл 'table3_significance_analysis.xlsx'")


ТАБЛИЦА 2: КОЭФФИЦИЕНТЫ РЕГРЕССИИ
      Признак  Линейная регрессия  Гребневая регрессия (α=3)  Lasso (α=120)
  sqft_living           58.158216                  58.896044      82.463985
     sqft_lot           -7.007353                  -7.028148      -7.269052
 waterfront_0       -47843.095359              -45978.061781  -28310.066303
   sqft_above           19.129167                  19.632248      -0.000000
     yr_built        -2544.576371               -2531.575022   -2469.699023
 yr_renovated            9.218884                   9.354153      10.876337
sqft_living15           67.091252                  67.650620      68.270147
   sqft_lot15           -1.978535                  -1.973250      -1.972264
   floors_1.0       -45549.314993              -40251.109297  -13676.258895
   floors_1.5       -32896.492286              -27291.143195      -0.000000
   floors_2.0       -33849.894652              -28374.222825    -517.403677
   floors_2.5        -3474.876851                2111